In [1]:
# load packages
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr
from utilities_spacial_vars import *

In [2]:
# s = pd.read_csv('gaa2_climate_2022.csv')

# load range map data
geo = gpd.read_file('../Data/GAA2/GAA2_allspecies.gdb', rows=20)

# load bioclimatic variables
bioclim = xr.open_dataset('../Data/ClimateVariables/bioclimatic_variables.nc')

In [3]:
i = 7
g = geo.iloc[[i]]

g.explore()

In [6]:
data = bioclim['MAT']

grid = GridCalc(data, g, 0.5)

var_window20y = grid.get_window(data.sel(year=slice(2000,2020)))
min_hist, max_hist = grid.calc_hist_limits(data.sel(year=slice(1940, 1970)), nsig=3)
nichefrac = grid.calc_fraction_in_niche(min_hist, max_hist, var_window20y)
print(nichefrac)

# visualize 
var_in_range = grid.data_window.where(grid.mask == 1)
var_in_range_flat = var_in_range.values.reshape(var_in_range.shape[0], -1)
_ = plt.plot(var_in_range.year, var_in_range_flat, color='gray', linewidth=0.5)
# plot dashed red lines for min_hist and max_hist
_ = plt.axhline(min_hist, color='red', linestyle='--')
_ = plt.axhline(max_hist, color='red', linestyle='--')


ValueError: zero-size array to reduction operation maximum which has no identity

In [117]:
var_in_range

<xarray.DataArray 'MAT' (year: 83, latitude: 5, longitude: 5)>
array([[[      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan, 3.2788067,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan]],

       [[      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan, 3.8726985,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan]],

       [[      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan, 4.524464 ,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan]],

       ...,

       [[      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan, 4.703839 ,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan]],

       [[      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan, 2.439446 ,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan]],

       [[      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan, 4.4441237,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan],
        [      nan,       nan,       nan,       nan,       nan]]],
      dtype=float32)
Coordinates:
  * longitude    (longitude) float32 -68.75 -68.5 -68.25 -68.0 -67.75
  * latitude     (latitude) float32 -27.75 -27.5 -27.25 -27.0 -26.75
    spatial_ref  int64 ...
  * year         (year) int64 1940 1941 1942 1943 1944 ... 2019 2020 2021 2022
    expver       int32 ...

In [ ]:
m